<a href="https://colab.research.google.com/github/nishant-jhaa/Attitude-Dynamics-and-Control-of-a-Nano-Satellite-Orbiting-Mars/blob/main/Orbit_Simulation_(Task_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

**Given Initial Conditions**

In [2]:
mars_R = 3396.19 #km
mars_mu = 42828.3 #km /s

#For LMO
altitude = 400 #km 
r_LMO = mars_R + altitude
theta_dot_LMO = np.sqrt(mars_mu/r_LMO**3) #rad/sec
t_LMO = 330 #s
angles_LMO = np.array([20,30,60])*np.pi/180 #Ω,i,θ(t_0), in radians
angles_LMO += np.array([0,0, theta_dot_LMO*t_LMO]) #Ω,i,θ(t), in radians

#For GMO
r_GMO = 20424.2 #km
theta_dot_GMO = np.sqrt(mars_mu/r_GMO**3) #rad/s
t_GMO = 1150 #s
angles_GMO = np.array([0,0,250])*np.pi/180 #Ω,i,θ(t_0), in radians
angles_GMO += np.array([0,0, theta_dot_GMO*t_GMO]) #Ω,i,θ(t), in radians

#For Spacecraft
sigma_BN = np.array([0.3, -0.4, 0.5])
w_BN_B = np.array([1.0, 1.75, -2.2])*np.pi/180
I = np.array([[10,0,0],[0,5,0],[0,0,7.5]])

In [5]:
def DCM_from_euler(x, seq):
   # This function computes the Direct Cosine Matrix (DCM) from a set of Euler angles and its rotational sequence
    
    for e in x:
        while e>2*np.pi:
            e -= 2*np.pi
    DCM_all = []
    
    for i,s in enumerate(seq):
        if s==3:
            # 3
            DCM_all.append(np.array([[ math.cos(x[i]), math.sin(x[i]),  0],
                   [ -math.sin(x[i]),  math.cos(x[i]), 0],
                   [ 0,  0,  1]]))
        elif s==2:
            # 2
            DCM_all.append(np.array([[ math.cos(x[i]), 0,  -math.sin(x[i])],
                   [ 0, 1, 0],
                   [ math.sin(x[i]),  0,  math.cos(x[i])]]))
        elif s==1:
            # 1
            DCM_all.append(np.array([[ 1, 0,  0],
                   [ 0,  math.cos(x[i]), math.sin(x[i])],
                   [ 0,  -math.sin(x[i]),  math.cos(x[i])]]))
    if len(seq)!=3:
        print("Kindly Recheck!!. Sequence does not have a length of 3.")
        return None
    final_DCM = DCM_all[2]@DCM_all[1]@DCM_all[0]
    return final_DCM

In [7]:
def inertial_r_r_dot(r, angle):
    
    # This Takes the norm of the position vector and the set of Euler angles and returns the position and velocity vector in the inertial frame
    
    mars_mu = 42828.3 #km /s    
    theta_dot = np.sqrt(mars_mu/r**3) #in rad/s
    
    NH = DCM_from_euler(angle, [3, 1, 3])
    r_H = np.array([r, 0, 0])
    r_inertial = NH.T@r_H
    
    r_dot = np.array([0, r*theta_dot, 0]) #[r, theta, h], in km/s
    r_dot_inertial = NH.T@r_dot
    return r_inertial, r_dot_inertial